In [2]:
import socket
import time
from datetime import datetime
import threading
import pyvisa
import numpy as np

In [6]:
'''
Setup communication
'''
# pip install pyvisa, pyvisa-py'
#find ID's with "lsUSB"
#then run this as super user
#echo 'SUBSYSTEM=="usb", ACTION=="add", ATTRS{idVendor}=="0957", ATTRS{idProduct}=="1798" GROUP="tty", MODE="0660"' >> /etc/udev/rules.d/99-com.rules
# sudo usermod -a -G tty pi

rm = pyvisa.ResourceManager()
rm.list_resources('?*')

inst = rm.open_resource('USB0::2391::6040::MY58103677::0::INSTR')

inst.write('*IDN?')
print(inst.read())

VisaIOError: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.

In [3]:
'''
set up backend
'''
import importlib
import serverBackend as sb
importlib.reload(sb)

#add the commands we want to use - see manual
osc_keys = [
    'CHANnel1:DISPlay',
    'CHANnel2:DISPlay',
    'CHANnel3:DISPlay',
    'CHANnel4:DISPlay',
    'DIGitize',
    'WAVeform:SOURce', # CHANnel1
    'WAVeform:FORMat',# BYTE
    'WAVeform:POINts', # 500
    'WAVeform:DATA', #?
    'RUN', #?
    'STOP', #?
    ]
sb.setup(inst, osc_keys)
sb.query('*IDN?')

'AGILENT TECHNOLOGIES,DSO-X 2014A,MY58103677,02.43.2018020635'

In [4]:
sb.set_single_param('CHANnel1:DISPlay','ON')
sb.set_single_param('CHANnel2:DISPlay','ON')
sb.set_single_param('CHANnel3:DISPlay','OFF')
sb.set_single_param('CHANnel4:DISPlay','OFF')
#sb.query('CHANnel1:DISPlay ON')

   set  CHANnel1:DISPlay: ON
   set  CHANnel2:DISPlay: ON
   set  CHANnel3:DISPlay: OFF
   set  CHANnel4:DISPlay: OFF


In [5]:
sb.set_single_param('WAVeform:SOURce','CHANnel1')
sb.set_single_param('WAVeform:FORMat','ASCii')
sb.set_single_param('WAVeform:POINts','100')
#sb.set_single_param('DIGitize','')
sb.set_single_param('RUN','')
#waveform = sb.query('WAVeform:DATA?')
#print(waveform)

   set  WAVeform:SOURce: CHANnel1
   set  WAVeform:FORMat: ASCii
   set  WAVeform:POINts: 100
   set  RUN: 


In [6]:
def get_C1C2():
    sb.set_single_param('DIGitize','')
    sb.set_single_param('WAVeform:SOURce','CHANnel1')
    ch1 = sb.query('WAVeform:DATA?')
    sb.set_single_param('WAVeform:SOURce','CHANnel2')
    ch2 = sb.query('WAVeform:DATA?')
    sb.set_single_param('RUN','')
    return ch1+'_'+ch2

def get_C1C2_avg():
    sb.set_single_param('DIGitize','')
    sb.set_single_param('WAVeform:SOURce','CHANnel1')
    ch1 = sb.query('WAVeform:DATA?')
    ch1 = np.array([float(s.strip(',')) for s in ch1.split(',')[1:]])    
    sb.set_single_param('WAVeform:SOURce','CHANnel2')
    ch2 = sb.query('WAVeform:DATA?')
    ch2 = np.array([float(s.strip(',')) for s in ch2.split(',')[1:]])
    sb.set_single_param('RUN','')
    return str(np.average(ch1))+', '+str(np.average(ch2))

sb.override_dict = {'C1C2?':get_C1C2, 'AvgC1C2?':get_C1C2_avg}
#sb12 = sb.query('C1C2?')
sb12 = sb.query('AvgC1C2?')
print(sb12)

   set  DIGitize: 
   set  WAVeform:SOURce: CHANnel1
   set  WAVeform:SOURce: CHANnel2
   set  RUN: 
0.9685269595959596, -0.14356833636363636


In [ ]:
serversocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = sb.get_ip() #"127.0.0.1"
port = 5007
serversocket.bind((host, port))
serversocket.listen(5)
print ('server started and listening')
while 1:
    clientsocket, address = serversocket.accept()
    sb.server(clientsocket, address)

server started and listening
